In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Imports

In [3]:
import logging

import pandas as pd

import core.config as cconfig
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import im_v2.common.data.client.historical_pq_clients as imvcdchpcl
import market_data as mdata

/app/core/statistics/random_samples.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
hdbg.init_logger(verbosity=logging.INFO)
log_level = logging.INFO

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-ced49272-fc0d-47f6-ab9f-9224d8c446d1.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.10.0' != container_version='1.12.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
(CVXPY) Dec 15 12:37:54 PM: Encountered unexpected exception importing solver GLPK:
ImportError("cannot import name 'glpk' from 'cvxopt' (/venv/lib/python3.9/site-packages/cvxopt/__init__.py)")
(CVXPY) Dec 15 12:37:54 PM: Encountered unexpected exception importing solver GLPK_MI:
ImportError("cannot import name 'glpk' from 'cvxopt' (/venv/lib/python3.9/site-packages/cvxopt/__init__.py)")
INFO  # Git
  branch_name='gp_scratch'
  hash='bc42b5cf5'
  # 

# Build config

In [5]:
config = {
    "start_ts": None,
    "end_ts": None,
    "wall_clock_time": pd.Timestamp("2100-01-01T00:00:00+00:00"),
    "columns": None,
    "columns_remap": None,
    "ts_col_name": "end_ts",
    "im_client": {
        "vendor": "bloomberg",
        "universe_version": "v1",
        "root_dir": "s3://cryptokaizen-data-test/v3/bulk",
        "partition_mode": "by_year_month",
        "dataset": "ohlcv",
        "contract_type": "spot",
        "data_snapshot": "",
        "download_mode": "manual",
        "downloading_entity": "",
        "aws_profile": "ck",
        "resample_1min": False,
        "version": "v1_0_0",
        "tag": "resampled_1min",
    },
}
config = cconfig.Config.from_dict(config)
print(config)

start_ts: None
end_ts: None
wall_clock_time: 2100-01-01 00:00:00+00:00
columns: None
columns_remap: None
ts_col_name: end_ts
im_client: 
  vendor: bloomberg
  universe_version: v1
  root_dir: s3://cryptokaizen-data-test/v3/bulk
  partition_mode: by_year_month
  dataset: ohlcv
  contract_type: spot
  data_snapshot: 
  download_mode: manual
  downloading_entity: 
  aws_profile: ck
  resample_1min: False
  version: v1_0_0
  tag: resampled_1min


# Load data

In [ ]:
im_client = imvcdchpcl.HistoricalPqByCurrencyPairTileClient(**config["im_client"])

In [ ]:
full_symbols = im_client.get_universe()
filter_data_mode = "assert"
actual_df = im_client.read_data(
    full_symbols,
    config["start_ts"],
    config["end_ts"],
    config["columns"],
    filter_data_mode,
)
hpandas.df_to_str(actual_df, num_rows=5, log_level=logging.INFO)

In [ ]:
asset_ids = im_client.get_asset_ids_from_full_symbols(full_symbols)
market_data = mdata.get_HistoricalImClientMarketData_example1(
    im_client,
    asset_ids,
    config["columns"],
    config["columns_remap"],
    wall_clock_time=config["wall_clock_time"],
)

In [ ]:
asset_ids = None
market_data_df = market_data.get_data_for_interval(
    config["start_ts"], config["end_ts"], config["ts_col_name"], asset_ids
)
hpandas.df_to_str(market_data_df, num_rows=5, log_level=logging.INFO)